In [1]:
import pandas as pd
import numpy as np
import geoenrich
from geoenrich import dataloader, enrichment, satellite, exports

In [12]:
output = exports.retrieve_data("plankton", 17, 'salinity3d', shape = 'buffer')

In [13]:
output

In [14]:
values = exports.export_to_array(output, 'salinity3d')

In [9]:
values